In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [4]:
# Load train data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Create a copy to use
X_train = train_data.copy()
y_train = train_data['Survived']

# Create Label Encoder
le = LabelEncoder()

# Modify the 'Sex' column directly in the DataFrame copy
X_train.loc[:, 'Sex'] = le.fit_transform(X_train['Sex'])

# Keep only the columns that are going to be used
X_train = X_train[['Pclass', 'Sex']]

# Create and train Naive-Bayes model
model = GaussianNB()
model.fit(X_train, y_train)



GaussianNB()

In [6]:
# Load test data
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
# Prepare test data
X_test = test_data.copy()
X_test.loc[:, 'Sex'] = le.transform(X_test['Sex'])  # Same label encoder
X_test = X_test[['Pclass', 'Sex']]

In [8]:
# Make predictions using the test data
predictions = model.predict(X_test)

In [9]:
# Internal validation
X = train_data[['Pclass', 'Sex']]
y = train_data['Survived']

# Codify LE
le = LabelEncoder()
X['Sex'] = le.fit_transform(train_data['Sex'])

# Dividing data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = GaussianNB()
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_val)

/tmp/ipykernel_33/1022756867.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'] = le.fit_transform(train_data['Sex'])


In [10]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_val, predictions)
print("Matriz de Confusão:")
print(conf_matrix)

Matriz de Confusão:
[[88 17]
 [22 52]]


In [12]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_val, predictions)
recall = recall_score(y_val, predictions)
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')

Precisão: 75.36%
Revocação: 70.27%


In [22]:
# Create DataFrame for submission
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Survived': predictions
})

# Save the DataFrame as a CSV for submission
submission.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' successfully created!")

Submission file 'submission.csv' successfully created!
